## Transfer Learning

<a href="https://colab.research.google.com/github/jgamero-ibrobotics/trabajo-pic/blob/Jesus/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install thop

In [1]:
import numpy as np
import thop
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchsummary import summary
from torch.utils.data import sampler
import torchvision.datasets as datasets
import torchvision.transforms as T
from torchvision import models
import matplotlib.pyplot as plt
import os
import zipfile
import random

In [ ]:
#Para que se importe de tu drive el zip del dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Descomprimir el zip
zip_path = 'drive/MyDrive/archive.zip'
extract_folder = 'data'

# Create the extraction folder if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

### Descargar datasets
Descarga  el dataset, le da el formato de resnet y crea los conjuntos de train, test y valid.

In [ ]:
DATA_PATH = "data/AugmentedAlzheimerDataset/"
MINIBATCH_SIZE = 32

train_transforms = T.Compose([T.Resize((224,224)),
                                    T.RandomRotation(60),
                                       T.RandomHorizontalFlip(),
                                       T.ToTensor(),
                                       T.Normalize([0.485, 0.456, 0.406],
                                                   [0.229, 0.224, 0.225])])

# Descargamos los datos de datapath
midataset = datasets.ImageFolder(DATA_PATH,
                                 transform=train_transforms)

train_size = int(0.9 * len(midataset))
val_size = int(0.04 * len(midataset))
test_size = len(midataset) - train_size - val_size
train_midataset, test_midataset, val_midataset = torch.utils.data.random_split(midataset, [train_size, test_size, val_size])

train_loader  = torch.utils.data.DataLoader(train_midataset,MINIBATCH_SIZE,shuffle=True)
val_loader  = torch.utils.data.DataLoader(val_midataset,MINIBATCH_SIZE,shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_midataset,MINIBATCH_SIZE,shuffle=False)

In [ ]:
for i, (x, y) in enumerate(val_loader):
    print(i, x.shape, y.shape)

### Usar GPU

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


### Mostrar imágenes

In [ ]:
def plot_random_images(dataset_path):
    classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
    num_images = 4
    
    # Plot images for each class
    for class_name in classes:
        class_folder = os.path.join(dataset_path, class_name)
        image_files = os.listdir(class_folder)
        random_images = random.sample(image_files, num_images)
        
        plt.figure(figsize=(12, 3))
        plt.suptitle(f'Random images from class: {class_name}', fontsize=14)
        
        for i, image_file in enumerate(random_images):
            image_path = os.path.join(class_folder, image_file)
            image = Image.open(image_path)
            
            plt.subplot(1, num_images, i + 1)
            plt.imshow(image)
            plt.axis('off')
        
        plt.show()

# Call the function with the path to your dataset folder
plot_random_images('data/AugmentedAlzheimerDataset/')

### Calcular Accuracy y Perdida
Función para probar el modelo sobre un conjunto de datos.

In [7]:
def test(model, loader): # funcion para calcular la precision del modelo
    num_correct = 0
    num_total = 0
    epoch_loss = 0.0
    model.eval() # ponemos el modelo en modo evaluacion
    model = model.to(device=device) # movemos el modelo al device (cpu o gpu)
    with torch.no_grad(): # deshabilitamos el calculo del gradiente (no necesita actualizar pesos)

        # xi: datos de entrada, yi: etiquetas
        for (xi, yi) in loader: # iteramos sobre los minibatches del loader de validacion o test 
            xi = xi.to(device=device, dtype = torch.float32) # movemos los datos al device (cpu o gpu) 
            yi = yi.to(device=device, dtype = torch.long) # movemos los datos al device (cpu o gpu)
            scores = model(xi) # mb_size, 10. corre el modelo sobre los datos de entrada
            loss =  F.cross_entropy(input=scores, target=yi)

            epoch_loss += loss.item()
            _, pred = scores.max(dim=1) #pred shape (mb_size). obtenemos la clase con mayor score (prediccion)   
            # comparamos las predicciones con las etiquetas y sumamos el numero de predicciones correctas
            num_correct += (pred == yi).sum() # pred shape (mb_size), yi shape (mb_size, 1).
            # sumamos 1 por cada prediccion correcta
            num_total += pred.size(0)


        loss_ = epoch_loss / len(loader)
        accuracy = float(num_correct)/num_total   

        return loss_, accuracy

### Cargar modelo pre-entrenado

In [30]:
model_resnet121 = models.densenet121(pretrained=True)

#### Exploremos el modelo

In [ ]:
for i, w in enumerate(model_resnet121.parameters()):
    print(i, w.shape, w.requires_grad)

In [ ]:
model_resnet121 = model_resnet121.to(device=device)
summary(model_resnet121, (3, 224, 224))
flops, params = thop.profile(model_resnet121, verbose=False,
                             inputs=(torch.randn(1, 3, 224, 224).to(device),))
(flops, params) = thop.clever_format([flops, params])
print(f'FLOPs: {flops}, params: {params}')

#### Ajustar a nuestro modelo
Eliminamos la ultima capa lineal dle modelo para añadir la nuestra con las 10 salidas que necesitamos segun las clases de nuestro dataset.

Ponemos el resto de parametros de la red a false para que no se ajustan en el entrenamiento.

In [35]:
for i, parameter in enumerate(model_resnet121.parameters()):
    parameter.requires_grad = False

for param in model_resnet121.features.denseblock4.parameters():#solo los de este bloque se ponen a True para que se modifiquen
    param.requires_grad =True  

In [ ]:
for i, parameter in enumerate(model_resnet121.parameters()):
    print(i, parameter.requires_grad) # los que son True se modifican, los que son False no

### Loop de entrenamiento

In [36]:
def train(model, optimizer, loader, lr_scheduler):
    
    for (xi, yi) in loader:
        model.train()
        xi = xi.to(device=device, dtype=torch.float32) 
        yi = yi.to(device=device, dtype=torch.long)
        scores = model(xi)
        loss = F.cross_entropy(input= scores, target=yi)
    
        optimizer.zero_grad()           
        loss.backward()
        optimizer.step()
    lr_scheduler.step()

In [37]:
lr = 0.0015
epochs = 10

model_resnet121.classifier=torch.nn.Sequential(
            torch.nn.Dropout(0.2,inplace=True),
            torch.nn.Linear(1024,4),)   

optimizer = torch.optim.Adam(model_resnet121.parameters(), lr=lr, betas=(0.9, 0.999))
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.85)

In [ ]:
model_resnet121 = model_resnet121.to(device=device)
summary(model_resnet121, (3, 224, 224))
flops, params = thop.profile(model_resnet121, verbose=False,
                             inputs=(torch.randn(1, 3, 224, 224).to(device),))
(flops, params) = thop.clever_format([flops, params])
print(f'FLOPs: {flops}, params: {params}')

In [ ]:
model1 = model_resnet121.to(device=device)
best_acc = 0.0
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []
lr_history = []

for epoch in range(epochs):

    train(model1, optimizer, train_loader, lr_scheduler)
    epoc_val_loss, epoc_val_accuracy = test(model1, val_loader)
    epoc_train_loss, epoc_train_accuracy = test(model1, train_loader)

    if best_acc <= epoc_train_accuracy:
        best_acc = epoc_train_accuracy
        best_model = copy.deepcopy(model1)

    train_loss_history.append(epoc_train_loss)
    train_acc_history.append(epoc_train_accuracy)   
    val_loss_history.append(epoc_val_loss)
    val_acc_history.append(epoc_val_accuracy) 
    lr_history.append(lr_scheduler.get_last_lr()[0])

    print(f'Epoch: {epoch} ACCURACY (validation): {epoc_val_accuracy:.4f}, ACCURACY (train): {epoc_train_accuracy:.4f},\
           LOSS (validation): {epoc_val_loss:.4f}, LOSS (train): {epoc_train_loss:.4f}')

In [ ]:
# Gráfica de la evolución de la pérdida y la precisión
plt.subplot(3, 1, 1)
plt.plot(train_loss_history, label='train')
plt.plot(val_loss_history, label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(train_acc_history, label='train')
plt.plot(val_acc_history, label='test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

plt.subplot(3, 1, 3)
plt.plot(lr_history)
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.tight_layout()
plt.show()

Volvemos a poner todos los parametros de la red a true, para realizar una segunda vuelta de entrenamiento y que asi conseguir que la red generalice mejor.

In [ ]:
for i, parameter in enumerate(best_model.parameters()):
    parameter.requires_grad = True

In [ ]:
best_model = best_model.to(device=device)
summary(best_model, (3, 224, 224))
flops, params = thop.profile(best_model, verbose=False,
                             inputs=(torch.randn(1, 3, 224, 224).to(device),))
(flops, params) = thop.clever_format([flops, params])
print(f'FLOPs: {flops}, params: {params}')

In [ ]:
best_model = best_model.to(device=device)
best_acc = 0.0
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []
lr_history = []

for epoch in range(epochs):

    train(best_model, optimizer, train_loader, lr_scheduler)
    epoc_val_loss, epoc_val_accuracy = test(best_model, val_loader)
    epoc_train_loss, epoc_train_accuracy = test(best_model, train_loader)

    if best_acc <= epoc_val_accuracy:
        best_acc = epoc_val_accuracy
        best_model_2 = copy.deepcopy(best_model)

    train_loss_history.append(epoc_train_loss)
    train_acc_history.append(epoc_train_accuracy)   
    val_loss_history.append(epoc_val_loss)
    val_acc_history.append(epoc_val_accuracy) 
    lr_history.append(lr_scheduler.get_last_lr()[0])

    print(f'Epoch: {epoch} ACCURACY (validation): {epoc_val_accuracy:.4f}, ACCURACY (train): {epoc_train_accuracy:.4f},\
           LOSS (validation): {epoc_val_loss:.4f}, LOSS (train): {epoc_train_loss:.4f}')

In [ ]:
test_loss, test_acc = test(best_model_2, test_loader)
print(f'ACCURACY (test): {test_acc:.4f}, LOSS (test): {test_loss:.4f}')

Parcticamos la inferencia sobre el modelo final.

In [ ]:
best_model_2.eval()
rnd_sample_idx = np.random.randint(len(test_loader))
xi = test_loader.dataset[rnd_sample_idx][0]
yi = test_loader.dataset[rnd_sample_idx][1]
xi = xi.to(device=device, dtype = torch.float32) # movemos los datos al device (cpu o gpu) 
print(f'La imagen muestreada representa un: {classes[test_loader.dataset[rnd_sample_idx][1]]}')
image = test_loader.dataset[rnd_sample_idx][0]
image = (image - image.min()) / (image.max() -image.min() )

scores = best_model_2(xi.unsqueeze(0))
_, pred = scores.max(dim=1)
print(f'La clase predicha es: {classes[pred]}')
plot_figure(image)